<h2>Solution Evaluation</h2>

This notebook contains some WIP work on solution evaluation.
<h3>One universal way to store solution from PDOTW, best_insertion and rasd</h3>
<h3>Given a solution, compute the cost</h3>

In [4]:
import pickle
import os, sys
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022'
sys.path.insert(1, src_dir_)

from src.util import read_pickle
from src.pdpt_route_schedule import MP_to_SP, calculate_SP_cost


DATA_DIR = "/home/tan/Documents/GitHub/pdpt_2022/toy"





<hr style="height:10px;">
<hr style="height:10px;">

<h4>Understand how Jason store data</h4>

In [ ]:
def understand_pdotw_sol(dir_):

    ini_sol_picke_filename = os.path.join(dir_, 'toy_initSol_all.pkl')
    ini_sol_pickle = read_pickle(ini_sol_picke_filename)
    ((created_truck_yCycle_total, created_truck_nCycle_total, created_truck_all_total, node_list_truck_hubs_total),
    (x_sol_total, y_sol_total, S_sol_total, D_sol_total, A_sol_total, Sb_sol_total, Db_sol_total, Ab_sol_total),
    (cost_cargo_size_value_total,  cost_cargo_number_value_total, cost_travel_value_total, cost_deviation_value_total),
    (truck_used_total, truck_route, cargo_delivered_total, cargo_undelivered_total, lb_truck, cargo_route, cargo_truck_total_all, cargo_in_truck_total)) = ini_sol_pickle


    ini_sol_txt_filename = os.path.join(dir_, 'toyinitSol.txt')
    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(ini_sol_txt_filename)

    res = {'truck_yCycle': list(created_truck_yCycle_total.keys()),
           'used_truck': truck_used_total,
           'truck_route': truck_route,
           'cargo_route': cargo_route,
           'x_sol': x_sol_total,
           'y_sol': y_sol_total,
           'S_sol': S_sol_total,
           'D_sol': D_sol_total,
           'A_sol': A_sol_total,
           'Sb_sol': Sb_sol_total,
           'Db_sol': Db_sol_total,
           'Ab_sol': Ab_sol_total,
          }

    filename = os.path.join(dir_, 'toy_inisol_clean.pkl')
    with open(filename, 'ab') as pickle_file:
        pickle.dump(res, pickle_file)

# understand_pdotw_sol(DATA_DIR)

<h4>Solution: save necessary information in pkl, ignore Jason's functions for storing and reading from txt file</h4>





<h4>Sanity check to see if the solutino saved in the format still produces the same cost </h4>

In [ ]:
def test_new_inisol_formate(dir_):

    def evaluate_sol_new_format(ins, dir_):
        ini_sol_filename = os.path.join(dir_, 'toy_inisol_clean.pkl')
        ini_sol = read_pickle(ini_sol_filename)

        truck_cost, travel_cost = eval_pdotw_sol(ins['constant'], ins['edge_shortest'],
                                                              ini_sol['used_truck'], ini_sol['truck_route'])
        print(truck_cost, travel_cost)

    def evaluate_sol_old_format(ins, dir_):
        ini_sol_txt_filename = os.path.join(dir_, 'toyinitSol.txt')
        truck_yCycle_file, truck_used_file, truck_route_file, \
        cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
        Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(ini_sol_txt_filename)

        truck_cost, travel_cost = eval_pdotw_sol(ins['constant'], ins['edge_shortest'],
                                                              truck_used_file, truck_route_file)

        print(truck_cost, travel_cost)


    toy_ins_filename = os.path.pdotw_solmat for Ini Sol')
    evaluate_sol_new_format(toy_ins, dir_)

# test_new_inisol_formate(DATA_DIR)


# ====== New data format for Ini Sol
# 90000.0 1281.5

<hr style="height:10px;">
<hr style="height:10px;">

<h4>Understand two different cost evaluation functions in Jason's code</h4>

Understand the differences between *Compute_cost_onTruck()* and *calculate_SP_cost()* 

In [ ]:
def eval_pdotw_sol_using_cpo_sp_cost_fun(dir_):

    ini_sol_filename = dir_ + '/toy_initSol.pkl'
    ini_sol_res = read_pickle(ini_sol_filename)

    pdpt_ins_filename = dir_ + '/toy.pkl'
    pdpt_ins = read_pickle(pdpt_ins_filename)
    constant = pdpt_ins['constant']
    cargo = pdpt_ins['cargo']
    truck = pdpt_ins['truck']

    edge_shortest = pdpt_ins['edge_shortest']


    truck_route = ini_sol_res['truck_route']
    print(truck_route)
    truck_used = ini_sol_res['used_truck']
    cargo_route = ini_sol_res['cargo_route']
    
    cargo_in, cargo_out, transfer_nodes = [], [], []
    selected_cargo = {}
    for cargo_key, cargo_value in cargo.items():
        if len(cargo_route[cargo_key])>0:
            selected_cargo[cargo_key] = cargo_value

    selected_node = []
    for cargo_value in selected_cargo.values():
        if cargo_value[3] not in selected_node:
            selected_node.append(cargo_value[3])
        if cargo_value[4] not in selected_node:
            selected_node.append(cargo_value[4])
    for truck_key in truck_used:
        truck_value = truck[truck_key]
        if truck_value[0] not in selected_node:
            selected_node.append(truck_value[0])
        if truck_value[1] not in selected_node:
            selected_node.append(truck_value[1])
    selected_edge = {}
    edges_ = list(set([(i,j) for i in selected_node for j in selected_node]))

    for i,j in edges_:
        selected_edge[(i,j)] = edge_shortest[(i,j)]

    # print(selected_edge)

    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
                      truck_used, truck_route, 
                      cargo_in, cargo_out, transfer_nodes)

    print(f'====== cost of pdpt solution')
    print(f' truck_cost {truck_cost}')
    print(f' travel_cost {travel_cost}')
    print(f' transfer_cost {transfer_cost}')

# eval_pdotw_sol_using_cpo_sp_cost_fun(DATA_DIR)

<h4>Conclusion: *Compute_cost_onTruck()* is equivalent to *calculate_SP_cost()* by letting cargo_in, cargo_out and transfer_nodes be empty lists.</h4>

<hr style="height:10px;">
<hr style="height:10px;">

<h4>Investigate how to convert pdotw solutions to pdpt solutions.</h4>

pdotw solution: x_sol, z_sol, y_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol<br>
pdpt solutions: MP: x_sol, s_sol, z_sol, y_sol, u_sol, D_sol, SP: g_sol, h_sol, D_sol


INCOMPLETE

Want to study this, because we need to embed some of the pdotw solutions into the MP and SP formulation when using RASD.

In [ ]:
def eval_pdotw_sol_using_CP_cost_function(dir_):

    class PDOTW_Sol():
        def __init__(self, res):
            self.truck_yCycle = res['truck_yCycle']
            self.used_truck = res['used_truck']
            self.truck_route = res['truck_route']
            self.cargo_route = res['cargo_route']
            self.x_sol = res['x_sol']
            self.y_sol = res['y_sol']
            self.S_sol = res['S_sol']
            self.D_sol = res['D_sol']
            self.A_sol = res['A_sol']
            self.Sb_sol = res['Sb_sol']
            self.Db_sol = res['Db_sol']
            self.Ab_sol = res['Ab_sol']

    ini_sol_res_filename = os.path.joint(dir_, 'toy_iniSol.pkl')
    ini_sol_res = read_pickle(ini_sol_res_filename)

    pdpt_ins_filename = dir_ + '/toy.pkl'
    pdpt_ins = read_pickle(pdpt_ins_filename)
    constant = pdpt_ins['constant']
    cargo = pdpt_ins['cargo']
    edge_shortest = pdpt_ins['edge_shortest']

    truck_nodes = ini_sol_res['truck_route']
    truck_MP = ini_sol_res['used_truck']
    cargo_route = ini_sol_res['cargo_route']
    selected_cargo = {}
    for cargo_key, cargo_value in cargo.items():
        if len(cargo_route[cargo_key])>0:
            selected_cargo[cargo_key] = cargo_value

    selected_node = []
    for cargo_value in selected_cargo.values():
        if cargo_value[3] not in selected_node:
            selected_node.append(cargo_value[3])
        if cargo_value[4] not in selected_node:
            selected_node.append(cargo_value[4])
    for truck_value in truck_MP.values():
        if truck_value[0] not in selected_node:
            selected_node.append(truck_value[0])
        if truck_value[1] not in selected_node:
            selected_node.append(truck_value[1])
    selected_edge = {}
    edges_ = list(set([(i,j) for i in selected_node for j in selected_node]))

    for i,j in edges_:
        selected_edge[(i,j)] = int(edge_shortest[(i,j)])


    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
        truck_MP, truck_nodes, 
        [], [], [])

In [ ]:
def convert_pdotw_sol_to_pdpt_sol(dir_): 
    ini_sol_res_filename = os.path.join(dir_, 'toy_initSol.pkl')
    pdotw_sol = read_pickle(ini_sol_res_filename)

    improve_sol_res_filename = os.path.join(dir_, 'toy_improve.pkl')
    pdpt_sol = read_pickle(improve_sol_res_filename)

    print('PDOTW sol')
    print(pdotw_sol.keys())

    print('PDPT_sol_MP')
    print(pdpt_sol['MP'].keys())

    print('PDPT_sol_SP')
    print(pdpt_sol['SP'].keys())

    print('PDPT_sol_route')
    print(pdpt_sol['route'].keys())

In [ ]:
convert_pdotw_sol_to_pdpt_sol(DATA_DIR)

<hr style="height:10px;">
<hr style="height:10px;">

<h4>Update solution after RASD</h4>

In [ ]:
from pdpt_rasd import select_subroutes, pdpt_route_schedule_decomposition


In [ ]:
def pdpt_rasd(dir_, verbose = 0):

    pdpt_ins = read_pickle(dir_ +'/toy.pkl', verbose = verbose-1) 

    pdpt_ins_ = read_pickle( '/home/tan/Documents/GitHub/pdpt_2022/data/case1.pkl', verbose = verbose-1)

    print(pdpt_ins['edge_shortest'])
    print(pdpt_ins_['edge_shortest'])


#     ini_sol_res_filename = dir_ + '/toy_initSol.pkl'
#     ini_sol_res = read_pickle(ini_sol_res_filename)

#     cargo_route = ini_sol_res['cargo_route']


#     subroutes = select_subroutes(pdpt_ins, cargo_route, verbose)

#     MP_sol, SP_sol, route_sol, costs = pdpt_route_schedule_decomposition(dir_+'/toy', pdpt_ins, ini_sol_res, subroutes, verbose = 0)

#     res = {'MP': {'x_sol': MP_sol[0],
#                   'x_sol': MP_sol[1],
#                   'z_sol': MP_sol[2],
#                   'y_sol': MP_sol[3],
#                   'u_sol': MP_sol[4],
#                   'D_sol': MP_sol[5],
#                  },
#             'SP':{'g_sol': SP_sol[0],
#                   'h_sol': SP_sol[1],
#                   'D_sol': SP_sol[-1],
#                  },
#             'route':{'truck_route': route_sol[0],
#                      'cargo_route': route_sol[-1],
#                     },
#             'cost':{'truck_cost': costs[0],
#                     'travel_cost': costs[1],
#                     'transfer_cost': costs[2],}
#             }
    
#     res_filename = dir_ + '/toyimprove.pkl'
#     with open(res_filename, 'wb') as pickle_file:
#         pickle.dump(res, pickle_file)

pdpt_rasd(DATA_DIR)

In [2]:
from util import manual_stop


def check_ini_sol(dir_):

    pdpt_ins_filename = os.path.join(dir_, 'toy.pkl')
    pdpt_ins = read_pickle(pdpt_ins_filename)
    edge_shortest = pdpt_ins['edge_shortest']
    node_list = pdpt_ins['nodes']
    truck_list = pdpt_ins['truck']

    ini_sol_res_filename = os.path.join(dir_, 'toy_initSol.pkl')
    ini_sol_res = read_pickle(ini_sol_res_filename)

    cargo_route = ini_sol_res['route']['cargo_route']
    for cargo_key, c_route in cargo_route.items():
        print(f'[{cargo_key}]: {c_route}')

    print('========================\n')
    truck_route = ini_sol_res['route']['truck_route']
    for truck_key, t_route in truck_route.items():
        print(f'[{truck_key}]: {t_route}')

    # Index k for truck is pre-defined
    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #y_sol: y^k_r, if parcel r is carried by truck k
    #s_sol: x^k_i, total size of cargos on truck k at node i
    #D_sol: D^k_i, depature time of truck k at node i
    #A_sol: A^k_i, arrival time of truck k at node i
    
    x_sol = ini_sol_res['MIP']['x_sol']



    truck_key = 'T2'
    n_curr = truck_list[truck_key][0]
    n_next = 10000
    # print(temp)
    # print(temp in edge_shortest.keys())
    # print(x_sol.keys())
    # print( (n_curr, n_curr, truck_key) in x_sol.keys())
    while n_next != truck_list[truck_key][1]:
        for n_next in node_list:
            if (n_curr, n_next) in edge_shortest.keys():
                print(f'{(n_curr, n_next)} in edge_shortest.keys()')
                if (n_curr, n_next, truck_key) in x_sol.keys():
                    print(f'{(n_curr, n_next, truck_key)} in x_sol.keys()')
            # if (n_curr, n_next) in edge_shortest.keys() and (n_curr, n_next, truck_key) in x_sol.keys():
                    i=0
                    if x_sol[(n_curr, n_next, truck_key)]== 1:

                        print(f'[{truck_key}: {n_curr, n_next}, x^k_ij = 1')
                        n_curr = n_next
                    i+=1
                    if i >100:
                        manual_stop()
        

check_ini_sol(DATA_DIR)

ModuleNotFoundError: No module named 'util'